In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Dash and Plotly components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

# Base64 encoding for image display
import base64

# Standard Python libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import python module
from animalShelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# Credentials for database access
username = "aacuser"
password = "aacuser123"

# Initialize CRUD operations with AnimalShelter class
db = AnimalShelter(username, password)

# Fetch data from MongoDB and create DataFrame
df = pd.DataFrame.from_records(db.read({}))

# Remove MongoDB _id column to prevent data table crash
if '_id' in df.columns:
    df.drop(columns=['_id'],inplace=True)
else:
    print("'_id' column not found in DataFrame")



#########################
# Dashboard Layout / View
#########################
# Initialize JupyterDash application
app = JupyterDash(__name__)

# Base64 encoded logo image for display
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Define dashboard layout using Dash components
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    # Header section with logo and title
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={'width':'100px', 'height':'100px'}),
        html.Center(html.B(html.H1('CS-340 Dashboard - Sara Elqaisi')))
        ], style={'display':'flex', 'align-items':'center'}),
    html.Hr(),
    # Filter section with dropdown and reset button
    html.Div([
        html.Label('Filter by Rescue Type:'),
        dcc.Dropdown(
            id='rescue-filter-dropdown',
            options=[
                {'label':'Water Rescue', 'value': 'Water Rescue'},
                {'label':'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label':'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue or Individual Tracking'},
            ],
            value=[],   # default value is empty so no filter is applied
            multi=False, # multiple selections set to false
        ),
        html.Button('Reset Filters', id='reset-button', n_clicks=0),
    ]),
    html.Hr(),
    # Data table section with additional features
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} 
                 for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action='native',
        row_selectable='single',
        selected_rows=[0]
    ),
    html.Br(),
    html.Hr(),
    # Side-by-side graph and map visualization
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])


#############################################
# Interaction Between Components / Controller
#############################################
# Update data table based on filter selection
@app.callback(Output('datatable-id','data'),
              [Input('rescue-filter-dropdown', 'value')])
def update_dashboard(rescue_type):
    # filter queries below
    if rescue_type == 'Water Rescue':
        filtered_data = df[(df['breed'].isin([
            'Labrador Retriever Mix',
            'Chesapeake Bay Retriever',
            'Newfoundland'
        ])) & (df['sex_upon_outcome'] == 'Intact Female') 
            & (df['age_upon_outcome_in_weeks'].between(26, 156))]
    elif rescue_type == 'Mountain or Wilderness Rescue':
        filtered_data = df[(df['breed'].isin([
            'Germen Shepherd',
            'Alaskan Malamute',
            'Old English Sheepdog',
            'Siberian Husky',
            'Rottweiler'
        ])) & (df['sex_upon_outcome'] == 'Intact Male') 
            & (df['age_upon_outcome_in_weeks'].between(26, 156))]
    elif rescue_type == 'Disaster Rescue or Individual Tracking':
        filtered_data = df[(df['breed'].isin([
            'Doberman Pinscher',
            'German Shepherd',
            'Golden Retriever',
            'Bloodhound',
            'Rottweiler'
        ])) & (df['sex_upon_outcome'] == 'Intact Male') 
            & (df['age_upon_outcome_in_weeks'].between(20, 300))]
    else:
        filtered_data = df
        
    return filtered_data.to_dict('records')


# Reset dropdown values using button click
@app.callback(
    Output('rescue-filter-dropdown','value'),
    [Input('reset-button','n_clicks')]
)
def reset_filters(n_clicks):
    return []


# Update pie chart based on data table selection
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return []
    
    df = pd.DataFrame.from_dict(viewData)
    
    breed_counts = df['breed'].value_counts()
    total_count = breed_counts.sum()
    under_2percent = breed_counts[breed_counts / total_count < 0.01]
    other_count = under_2percent.sum()
    breed_counts = breed_counts.drop(under_2percent.index)
    breed_counts["Other"] = other_count
    
    pie_chart = dcc.Graph(
        figure=px.pie(
            names=breed_counts.index.tolist(),
            values=breed_counts.values.tolist(),
            title='Breeds of Animals'
        ),
        id='breed-pie-chart'
    )
    return pie_chart
    
    
# Highlight selected row in data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    else:
        return [{
            'if': { 'column_id': i },
            'background_color': '#D2F3FF'
        } for i in selected_columns]


# Update map location based on selected row in data table
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

# Run the app in debug mode
app.run_server(debug=True)


Connection Successful
Dash app running on http://127.0.0.1:10157/
